<h1> Parameter Tunning 
    
    Look at epochs duration
    
    (6 months of data used)

# Importing libraries:

In [2]:
!pip install pandas
!pip install sklearn

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Import data; set X and y; fill nan values and split in test, training and validation data:

In [3]:

# import data
data = pd.read_csv('Data_set_1_smaller.csv', index_col = 0)

# for later use
features_num = 15

# 2018 data
data = data.loc[data.index > 2018070000, :]

# reset index
data.reset_index(inplace = True)
data.drop('index', axis = 1, inplace = True)

# fill nan values
data.fillna(method = 'ffill', inplace = True)

# divide features and labels
X = data.iloc[:, 0:15] .values # turns it into an array
y = data.loc[:, 'Offers'].values # turns it into an array

from sklearn.model_selection import train_test_split

# divide data into train and test 
X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size = 0.15, shuffle=False)

# divide data into train and validation
X_train, X_val, y_train, y_val = train_test_split(
         X_train, y_train, test_size = 0.3, shuffle=False)

# Apply feature scaling:

In [4]:
from sklearn.preprocessing import MinMaxScaler

# feature scaling 
sc_X = MinMaxScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

# Importing the Keras libraries and packages:

In [5]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import LeakyReLU
from keras import initializers
from keras import optimizers
from keras.callbacks import EarlyStopping
from sklearn.model_selection import ParameterGrid
import time

mae_cv = []
mse_cv = []
mae_gen = []
mse_gen  =[]
rmse_gen = []
mae_nor = []
mae_spi = []
mse_nor = []
mse_spi = []
rmse_nor = []
rmse_spi = []
time_count = []

# Prepare of data according to LSTM needs,  create regressor & tune:

In [6]:
# parameters
steps = 96
n_hidden = 1
units = 50
batch_size = 96

# function to split data into correct shape for RNN
def split_data(X, y, steps):
    X_, y_ = list(), list()
    for i in range(steps, len(y)):
        X_.append(X[i - steps : i, :])
        y_.append(y[i]) 
    return np.array(X_), np.array(y_)

X_train, y_train = split_data(X_train, y_train, steps)
X_test, y_test = split_data(X_test, y_test, steps)
X_val, y_val = split_data(X_val, y_val, steps)

# function to cut data set so it can be divisible by the batch_size
def cut_data(data, batch_size):
     # see if it is divisivel
    condition = data.shape[0] % batch_size
    if condition == 0:
        return data
    else:
        return data[: -condition]

# empty list to append metric values
mae_cv = []
mse_cv = []
mae_gen = []
mse_gen  =[]
rmse_gen = []
mae_nor = []
mae_spi = []
mse_nor = []
mse_spi = []
rmse_nor = []
rmse_spi = []

# Do tunning of epochs to understand how long it takes:

In [7]:
epochs_range = [50, 75, 100, 125, 150, 200, 400]

for i in epochs_range:
    start_time = time.time()
    # design the LSTM
    def regressor_tunning(kernel_initializer = 'he_normal',
                          bias_initializer = initializers.Ones()):
        model = Sequential()
        model.add(LSTM(units = units,                    
                       batch_input_shape = (batch_size, steps, features_num), 
                       stateful = True,
                       return_sequences = True,
                       kernel_initializer = kernel_initializer,
                       bias_initializer = bias_initializer))
        model.add(LeakyReLU(alpha = 0.2))
        model.add(Dropout(0.2))
        for layer in list(reversed(range(n_hidden))):
            if layer == 0:
                model.add(LSTM(units = units, 
                               kernel_initializer = kernel_initializer,
                               bias_initializer = bias_initializer))
                model.add(LeakyReLU(alpha = 0.2))
                model.add(Dropout(0.2))
            else:
                model.add(LSTM(units = units, 
                               batch_input_shape = (batch_size, steps, features_num), 
                               stateful = True,
                               return_sequences = True,
                               kernel_initializer = kernel_initializer,
                               bias_initializer = bias_initializer))
                model.add(LeakyReLU(alpha = 0.2))
                model.add(Dropout(0.2))
        model.add(Dense(1, activation='linear'))
        model.compile(loss = 'mse', metrics = ['mse', 'mae'], optimizer = 'Adamax')
        return model

    model = regressor_tunning()

    # apply patience callback
    early_stopping = EarlyStopping(monitor='mse', patience=10)

    # fitting the LSTM to the training set
    history = model.fit(cut_data(X_train, batch_size),
                        cut_data(y_train, batch_size), 
                        batch_size = batch_size, 
                        epochs = i,
                        shuffle = False, 
                        validation_data = (cut_data(X_val, batch_size), cut_data(y_val, batch_size)),
                        callbacks = early_stopping)

    X_test = cut_data(X_test, batch_size)
    y_test = cut_data(y_test, batch_size)

    # make new predicitons with test set
    y_pred = model.predict(cut_data(X_test, batch_size), batch_size = batch_size)

    # plot the training progression
    #plt.plot(history.history['val_loss'], label = 'train')
    #plt.plot(history.history['val_loss'], label = 'test')
    #plt.legend()
    #plt.show()

    from sklearn.metrics import mean_squared_error as mse
    from sklearn.metrics import mean_absolute_error as mae

    rmse_error = mse(y_test, y_pred, squared = False)
    mse_error = mse(y_test, y_pred) # 1479.61335
    mae_error = mae(y_test, y_pred) # 23.1525

    rmse_gen.append(rmse_error)
    mse_gen.append(mse_error)
    mae_gen.append(mae_error)

    # =============================================================================
    # Metrics evaluation on spike regions
    # =============================================================================

    y_spike_occ = pd.read_csv('Spike_binary_1std.csv', usecols = [6])

    # create array same size as y_test
    y_spike_occ = y_spike_occ.iloc[- len(y_test):]
    y_spike_occ = pd.Series(y_spike_occ.iloc[:,0]).values


    # smal adjustment
    y_test = pd.Series(y_test)
    y_test.replace(0, 0.0001,inplace = True)


    # select y_pred and y_test only for regions with spikes
    y_test_spike = (y_test.T * y_spike_occ).T
    y_pred_spike = (y_pred.T * y_spike_occ).T
    y_test_spike = y_test_spike[y_test_spike != 0]
    y_pred_spike = y_pred_spike[y_pred_spike != 0]

    # calculate metric
    rmse_spike = mse(y_test_spike, y_pred_spike, squared = False)
    mse_spike = mse(y_test_spike, y_pred_spike)
    mae_spike = mae(y_test_spike, y_pred_spike)

    rmse_spi.append(rmse_spike)
    mse_spi.append(mse_spike)
    mae_spi.append(mae_spike)

    # =============================================================================
    # Metric evaluation on normal regions
    # =============================================================================

    # inverse y_spike_occ so the only normal occurences are chosen
    y_normal_occ = (y_spike_occ - 1) * (-1)

    # sanity check
    y_normal_occ.sum() + y_spike_occ.sum() # gives the correct total 

    # select y_pred and y_test only for normal regions
    y_test_normal = (y_test.T * y_normal_occ).T
    y_pred_normal = (y_pred.T * y_normal_occ).T
    y_test_normal = y_test_normal[y_test_normal != 0.00]
    y_pred_normal = y_pred_normal[y_pred_normal != 0.00]

    # calculate metric
    rmse_normal = mse(y_test_normal, y_pred_normal, squared = False)
    mse_normal = mse(y_test_normal, y_pred_normal)
    mae_normal = mae(y_test_normal, y_pred_normal)

    rmse_nor.append(rmse_normal)
    mse_nor.append(mse_normal)
    mae_nor.append(mae_normal)

    elapsed_time = time.time() - start_time

    time_count.append(elapsed_time)

Epoch 1/50
53/53 [==============================] - 14s 266ms/step - loss: 14039.4902 - mse: 14039.4902 - mae: 110.3462 - val_loss: 15014.0674 - val_mse: 15014.0674 - val_mae: 110.8759
Epoch 2/50
53/53 [==============================] - 13s 251ms/step - loss: 13447.7627 - mse: 13447.7627 - mae: 107.5981 - val_loss: 14652.0381 - val_mse: 14652.0371 - val_mae: 109.2327
Epoch 3/50
53/53 [==============================] - 12s 224ms/step - loss: 13113.4424 - mse: 13113.4424 - mae: 106.0417 - val_loss: 14344.4912 - val_mse: 14344.4912 - val_mae: 107.8131
Epoch 4/50
53/53 [==============================] - 11s 214ms/step - loss: 12800.2080 - mse: 12800.2080 - mae: 104.5661 - val_loss: 14060.7285 - val_mse: 14060.7285 - val_mae: 106.4845
Epoch 5/50
53/53 [==============================] - 11s 211ms/step - loss: 12526.1709 - mse: 12526.1709 - mae: 103.2208 - val_loss: 13797.1738 - val_mse: 13797.1738 - val_mae: 105.2360
Epoch 6/50
53/53 [==============================] - 12s 225ms/step - loss: 

53/53 [==============================] - 11s 204ms/step - loss: 4548.2227 - mse: 4548.2227 - mae: 51.7799 - val_loss: 6394.0713 - val_mse: 6394.0713 - val_mae: 60.5734
Epoch 47/50
53/53 [==============================] - 11s 208ms/step - loss: 4446.4536 - mse: 4446.4536 - mae: 50.7424 - val_loss: 6279.9536 - val_mse: 6279.9536 - val_mae: 59.6232
Epoch 48/50
53/53 [==============================] - 11s 207ms/step - loss: 4334.7729 - mse: 4334.7729 - mae: 49.8467 - val_loss: 6168.7852 - val_mse: 6168.7852 - val_mae: 58.6838
Epoch 49/50
53/53 [==============================] - 11s 203ms/step - loss: 4220.5308 - mse: 4220.5308 - mae: 48.5890 - val_loss: 6061.3169 - val_mse: 6061.3169 - val_mae: 57.7612
Epoch 50/50
53/53 [==============================] - 11s 207ms/step - loss: 4098.2803 - mse: 4098.2803 - mae: 47.4804 - val_loss: 5955.7329 - val_mse: 5955.7329 - val_mae: 56.8402


FileNotFoundError: [Errno 2] File Spike_binary_1std.csv does not exist: 'Spike_binary_1std.csv'

# Save results into a data frame

In [ ]:
# Save
results = pd.DataFrame({'rmse_general': rmse_gen, 
                 
                        'mae_general': mae_gen,
                        
                        'rmse_spike': rmse_spi,
                 
                        'mae_spike': mae_spi,
                        
                        'rmse_normal': rmse_nor,
                    
                        'mae_normal': mae_nor,
                       
                        'time': time_count}, index = epochs_range)

